In [1]:
%reload_ext autoreload
%autoreload 2


#This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
import re
sys.path.append('/Users/rikhoekstra/develop/republic_clean/')

from collections import defaultdict
import pandas as pd
from collections import Counter, defaultdict
import json
import republic.model.republic_document_model as rdm

repo_dir = os.path.split(os.getcwd())[0]

if repo_dir not in sys.path:
    sys.path.append(repo_dir)
    
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
from republic.elastic.republic_elasticsearch import initialize_es

rep_es = initialize_es(host_type='external', timeout=60)

In [10]:
# N.B change the seed for a different selection!
# and add used seeds to the used_seeds variable
from random import randint
import json
def get_seed():
    with open('./used_seeds.json','r') as infl:
        used_seeds = json.load(infl)
    seed = randint(0,1000000)
    if seed not in used_seeds:
        used_seeds.append(seed)
        result = seed
        with open('./used_seeds.json','w') as outfl:
            json.dump(used_seeds, outfl, ensure_ascii=False)
        return result

In [12]:
seed = None
while seed==None:
    seed = get_seed()


29745


In [13]:

query = {
    'function_score': {
        'query': {
            'bool': {
                'must': [
                    {'match': {'type': 'resolution'}},
                    {'range': {'metadata.session_year': {'gte': 1700, 'lte': 1797}}}
                ]
            }            
        },
        'random_score': {
            'seed': seed,
            'field': '_seq_no'
        }
    }
}

In [15]:
response = rep_es.retrieve_resolutions_by_query(query=query, size=500)

using query param
resolutions


In [16]:
len(response)

500

In [17]:
df = pd.read_csv('/Users/rikhoekstra/develop/tagging/inception_input/tag_resoluties.csv', sep='\t', index_col=0)
fls = list(df.resolutie)

In [18]:
len( [r.id for r in response if r.id not in fls])

497

In [19]:
outdir = './inception_input/20230109'

In [20]:
for r in response:
    fl = r.id
    with open(os.path.join(outdir, fl +'.txt'),'w') as outfl:
        txt = '\n'.join( [p.text for p in r.paragraphs])
        outfl.write(txt)

In [21]:

ndf = pd.DataFrame([r.id for r in response], columns=['resolutie'])
df.append(ndf)

,resolutie
0,session-1709-07-11-ordinaris-num-1-resolution-66
1,session-1785-04-19-ordinaris-num-1-resolution-31
2,session-1723-02-25-ordinaris-num-1-resolution-6
3,session-1778-06-22-ordinaris-num-1-resolution-5
4,session-1717-03-26-ordinaris-num-1-resolution-6
...,...
495,session-1722-12-08-ordinaris-num-1-resolution-5
496,session-1725-07-26-ordinaris-num-1-resolution-1
497,session-1709-04-22-ordinaris-num-1-resolution-13
498,session-1705-10-31-ordinaris-num-1-resolution-14


In [22]:
df.to_csv('/Users/rikhoekstra/develop/tagging/inception_input/tag_resoluties.csv', sep='\t')